In [46]:
from pandas import read_csv
from os import walk, path
from re import sub
from pycorenlp import StanfordCoreNLP
from pickle import dump, load

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

ps = PorterStemmer()
stop = stopwords.words('english')

# Cut keywords and references

In [2]:
langs = read_csv('../prepared-data/texts/detected_languages.tsv', sep='\t')

In [3]:
def get_lang(num):
    return langs['Lang'][int(num)-1]

In [30]:
eng_files = []

for a, b, files in walk(path.join('..', 'prepared-data', 'texts')):
    for file in files:
        if file.split('.')[1] != 'txt':
            continue
        if get_lang(file.split('.')[0]) == 'en':
            with open(a + '/' + file, 'r') as f:
                eng_files.append(f.read())

In [5]:
kw_marker_1 = 'Ключевые  слова'
kw_marker_5 = 'Ключевые  слова'
kw_marker_2 = 'Key words'
kw_marker_3 = 'Ключевые слова'
kw_marker_4 = 'Keywords'
intro_marker_1 = 'Introduction'
intro_marker_2 = 'INTRODUCTION'
ref_marker_1 = 'References'
ref_marker_2 = 'REFERENCES'

eng_files_cleared = []

for num, eng_file in enumerate(eng_files):
    i = eng_file
    if eng_file.split(' ', 1)[0] == 'Keywords:':
        i = eng_file.split('\n\n', 1)[1]
    temp = i
    if kw_marker_2 in i:
        temp = i.split(kw_marker_2)[1].split('\n\n', 1)[1]
    elif kw_marker_4 in i:
        temp = i.split(kw_marker_4)[1].split('\n\n', 1)[1]
    else:
        temp = i
    cleartemp = temp
    if kw_marker_1 in temp:
        cleartemp  = i.split(kw_marker_1)[1].split('\n\n', 1)[1]
    elif kw_marker_3 in temp:
        cleartemp  = i.split(kw_marker_3)[1].split('\n\n', 1)[1]
    else:
        cleartemp  = temp
    cleartemp = cleartemp.split(ref_marker_1)[0]
    cleartemp = cleartemp.split(ref_marker_1)[0]
    if cleartemp.find(intro_marker_1) > 5000:
        eng_files_cleared.append(str(cleartemp))
        continue
    if intro_marker_1 in cleartemp:
        eng_files_cleared.append(cleartemp.split(intro_marker_1)[1].split('\n\n', 1)[1])
    elif intro_marker_2 in cleartemp:
        eng_files_cleared.append(cleartemp.split(intro_marker_2)[1].split('\n\n', 1)[1])
    else:
        eng_files_cleared.append(str(cleartemp))

In [10]:
for a, i in enumerate(eng_files_cleared):
    with open('../prepared-data/texts-without-refs-and-keywords/' + str(a) + '.txt', 'w') as f:
        f.write(i)

FileNotFoundError: [Errno 2] No such file or directory: '../prepared-data/texts-without-refs-and-keywords/0.txt'

# Make alpha

In [33]:
eng_files_alpha = []

for i in eng_files_cleared:
    text = sub('http\S+|www\S+', '', i)
    eng_files_alpha.append(sub('\.+', '.', sub(' \.', '.', sub(' +',' ', sub(r'[^,.a-zA-Z -]+', '', text.replace('-\n', '').replace('\n', ' '))))))

In [36]:
for a, i in enumerate(eng_files_alpha):
    with open('../prepared-data/alpha/' + str(a) + '.txt', 'w') as f:
        f.write(i)

In [47]:
with open('alpha', 'rb') as f:
    eng_files_alpha = load(f)

## Lemmatize

In [57]:
eng_files_stemmed = []

for file in eng_files_alpha:
    eng_files_stemmed.append(sub('\,+', ',', sub('\.+', '.', sub(' +\.', '.', sub(' +\,', ',', sub(' +',' ', ' '.join([ps.stem(word) for word in word_tokenize(file) if word not in stop])))))))

In [62]:
with open('stemmed', 'wb') as f:
    dump(eng_files_stemmed, f)

In [56]:
sub(' +\.', '.','these linguist unit usual treat restrict co-occurr two syntact bound element kilgariff . ggg')

'these linguist unit usual treat restrict co-occurr two syntact bound element kilgariff. ggg'